# Data Preprocessing for predicting Transcription Rate (TS)

This notebook is ment to convert raw cell data from several wells into multichannel images (along with its corresponding metadata).

Data was taken from:
`/storage/groups/ml01/datasets/raw/20201020_Pelkmans_NascentRNA_hannah.spitzer/` and server `vicb-submit-01`. 

Load libraries and set Directories paths:

In [1]:
import numpy as np
import pandas as pd
# To display all the columns
pd.options.display.max_columns = None
import os
import sys
import matplotlib.pyplot as plt
import json

# Set paths
BASE_DIR = os.path.realpath(os.path.join(os.path.abspath(''),'../..'))
if not os.path.exists(BASE_DIR):
    raise Exception('Base path {} does not exist!'.format(BASE_DIR))
else:
    print('BASE_DIR: {}'.format(BASE_DIR))
    
# Add BASE_DIR to sys paths (for loading libraries)
sys.path.insert(1, os.path.join(BASE_DIR, 'workspace'))
# Load mpp_data library to convert raw data into images
from pelkmans.mpp_data import MPPData as MPPData
    
PARAMETERS_FILE = os.path.join(BASE_DIR, 'workspace/scripts/Parameters/temp_parameters.json')
if not os.path.exists(PARAMETERS_FILE):
    raise Exception('Parameter file {} does not exist!'.format(PARAMETERS_FILE))


BASE_DIR: /home/hhughes/Documents/Master_Thesis/Project
Setting BASE_DIR to /home/hhughes/Documents/Master_Thesis/Project


Open parameters file

In [2]:
# Open parameters
with open(PARAMETERS_FILE) as params_file:
    p = json.load(params_file)
#del(p['_comment'])

In [3]:
p.keys()

dict_keys(['_comment', 'raw_data_dir', 'dir_type', 'perturbations_and_wells', 'output_data_dir', 'seed', 'input_channels', 'output_channels', 'aggregate_output', 'train_frac', 'val_frac', 'img_size', 'subtract_background', 'background_value', 'normalise', 'percentile', 'add_cell_cycle_to_metadata', 'cell_cycle_file', 'add_well_info_to_metadata', 'well_info_file', 'filter_criteria', 'filter_values', 'convert_into_image', 'remove_original_data', 'project_into_scalar', 'method', 'parameter_file_name'])

In [4]:
p

{'_comment': 'Save file name as reference ---------------------------------',
 'raw_data_dir': '/home/hhughes/Documents/Master_Thesis/Project/datasets/raw',
 'dir_type': 'hannah',
 'perturbations_and_wells': {'184A1_hannah_unperturbed': ['I09']},
 'output_data_dir': '/home/hhughes/Documents/Master_Thesis/Project/datasets/184A1_hannah_EU_regression_TEST_2',
 'seed': 42,
 'input_channels': ['00_DAPI',
  '07_H2B',
  '01_CDK9_pT186',
  '03_CDK9',
  '05_GTF2B',
  '07_SETD1A',
  '08_H3K4me3',
  '09_SRRM2',
  '10_H3K27ac',
  '11_KPNA2_MAX',
  '12_RB1_pS807_S811',
  '13_PABPN1',
  '14_PCNA',
  '15_SON',
  '16_H3',
  '17_HDAC3',
  '19_KPNA1_MAX',
  '20_SP100',
  '21_NCL',
  '01_PABPC1',
  '02_CDK7',
  '03_RPS6',
  '05_Sm',
  '07_POLR2A',
  '09_CCNT1',
  '10_POL2RA_pS2',
  '11_PML',
  '12_YAP1',
  '13_POL2RA_pS5',
  '15_U2SNRNPB',
  '18_NONO',
  '20_ALYREF',
  '21_COIL'],
 'output_channels': ['00_EU'],
 'aggregate_output': 'avg',
 'train_frac': 0.8,
 'val_frac': 0.1,
 'img_size': 224,
 'subtract

Set raw data directory

In [5]:
DATA_DIR = p['raw_data_dir']
if not os.path.exists(DATA_DIR):
    raise Exception('Data path {} does not exist!'.format(DATA_DIR))
else:
    print('DATA_DIR: {}'.format(DATA_DIR))

DATA_DIR: /home/hhughes/Documents/Master_Thesis/Project/datasets/raw


Check available data (Perturbations and Wells):

In [6]:
# Save available local Perturbations and Wells
perturbations = [per for per in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, per))]
local_data = {}
#print('Local available perturbations-wells:\n')
for per in perturbations:
    pertur_dir = os.path.join(DATA_DIR, per)
    wells = [w for w in os.listdir(pertur_dir) if os.path.isdir(os.path.join(pertur_dir, w))]
    #print('{}\n\t{}\n'.format(p, wells))
    local_data[per] = wells

Select Perturbations and its wells to process: 

In [7]:
print('Local available perturbations-wells:\n{}'.format(local_data))

# In case you only want to load some specific perturbations and/or wells:
selected_data = {
    '184A1_hannah_unperturbed': ['I11', 'I09'],
    '184A1_hannah_TSA': ['J20', 'I16'],
}

selected_data = p['perturbations_and_wells']

# Process all available data:
#selected_data = local_data

print('\nSelected perturbations-wells:\n{}'.format(selected_data))

#Generate and save data dirs
data_dirs = []
for per in selected_data.keys():
    for w in selected_data[per]:
        d = os.path.join(DATA_DIR, per, w)
        data_dirs.append(d)
        if not os.path.exists(d):
            raise Exception('{} does not exist!\nCheck if selected_data contain elements only from local_data dict.'.format(d))
p['data_dirs'] = data_dirs

Local available perturbations-wells:
{'184A1_hannah_unperturbed': ['I11', 'I09', 'J10'], '184A1_hannah_TSA': ['J20', 'I16', 'J13']}

Selected perturbations-wells:
{'184A1_hannah_unperturbed': ['I09']}


Process data:

In [8]:
for data_dir in p['data_dirs']:
    print('\nProcessing dir {}...'.format(data_dir))
    # Load data as an MPPData object
    mpp_temp = MPPData.from_data_dir(data_dir,
                                     dir_type=p['dir_type'],
                                     seed=p['seed'])
    
    # Add cell cycle to metadata (G1, S, G2)
    # Important! If mapobject_id_cell is not in cell_cycle_file =>
    # its corresponding cell is in Mitosis phase!
    if p['add_cell_cycle_to_metadata']:
        print('Adding cell cycle to metadata...')
        mpp_temp.add_cell_cycle_to_metadata(os.path.join(DATA_DIR, p['cell_cycle_file']))
    
    # Add well info to metadata
    if p['add_well_info_to_metadata']:
        print('Adding well info to metadata...')
        mpp_temp.add_well_info_to_metadata(os.path.join(DATA_DIR, p['well_info_file']))
    
    # Remove unwanted cells
    if p.get('filter_criteria', None) is not None:
        print('Removing unwanted cells...')
        mpp_temp.filter_cells(p['filter_criteria'], p['filter_values'])

    # Subtract background  values for each channel
    if p['subtract_background']:
        print('Subtracting background...')
        mpp_temp.subtract_background(os.path.join(DATA_DIR, p['background_value']))
    
    # Project every uni-channel images into a scalar for further analysis
    if p['project_into_scalar']:
        print('\nProjecting data...')
        mpp_temp.add_scalar_projection(p['method'])
        
    # Split data into train, validation and test
    train_temp, val_temp, test_temp = mpp_temp.train_val_test_split(p['train_frac'], p['val_frac'])
    del(mpp_temp)
    
    if p['convert_into_image']:
        print('Converting data into images...')
        train_temp.add_image_and_mask(data='MPP', remove_original_data=p['remove_original_data'], img_size=p['img_size'])
        val_temp.add_image_and_mask(data='MPP', remove_original_data=p['remove_original_data'], img_size=p['img_size'])
        test_temp.add_image_and_mask(data='MPP', remove_original_data=p['remove_original_data'], img_size=p['img_size'])
    
    # Concatenate wells
    # Check first if data sets are already defined
    if 'train' not in globals().keys():
        train, val, test = train_temp, val_temp, test_temp
    else:
        print('Merging data with loaded well...')
        val.merge_instances([val_temp])
        del(val_temp)
        test.merge_instances([test_temp])
        del(test_temp)
        train.merge_instances([train_temp])
        del(train_temp)

# Normalize train, val and test using the normalization parameters
# got from the train data (inner percentile% of train data)
if p['normalise']:
    print('\nNormalizing data...')
    rescale_values = train.rescale_intensities_per_channel(percentile=p['percentile'], )
    _ = val.rescale_intensities_per_channel(rescale_values=rescale_values)
    _ = test.rescale_intensities_per_channel(rescale_values=rescale_values)
    p['normalise_rescale_values'] = list(rescale_values)


Processing dir /home/hhughes/Documents/Master_Thesis/Project/datasets/raw/184A1_hannah_unperturbed/I09...
Adding cell cycle to metadata...
Adding well info to metadata...
Removing unwanted cells...
Total number of cells: 886
269 cells cutted by filter: is_border_cell == 1
30 cells cutted by filter: is_polynuclei_184A1 == 1
323 cells cutted by filter: cell_cycle == NaN
Number of cutted cells: 323


missing background value for channels ['00_EU', '09_SRRM2_ILASTIK', '15_SON_ILASTIK']


Subtracting background...

Projecting data...

Normalizing data...


In [9]:
train.metadata

,mapobject_id,plate_name,well_name,well_pos_y,well_pos_x,tpoint,zplane,label,is_border,mapobject_id_cell,plate_name_cell,well_name_cell,well_pos_y_cell,well_pos_x_cell,tpoint_cell,zplane_cell,label_cell,is_border_cell,is_mitotic,is_mitotic_labels,is_polynuclei_HeLa,is_polynuclei_HeLa_labels,is_polynuclei_184A1,is_polynuclei_184A1_labels,cell_cycle,cell_type,perturbation,duration,00_DAPI_avg,07_H2B_avg,01_CDK9_pT186_avg,03_CDK9_avg,05_GTF2B_avg,07_SETD1A_avg,08_H3K4me3_avg,09_SRRM2_avg,10_H3K27ac_avg,11_KPNA2_MAX_avg,12_RB1_pS807_S811_avg,13_PABPN1_avg,14_PCNA_avg,15_SON_avg,16_H3_avg,17_HDAC3_avg,19_KPNA1_MAX_avg,20_SP100_avg,21_NCL_avg,01_PABPC1_avg,02_CDK7_avg,03_RPS6_avg,05_Sm_avg,07_POLR2A_avg,09_CCNT1_avg,10_POL2RA_pS2_avg,11_PML_avg,12_YAP1_avg,13_POL2RA_pS5_avg,15_U2SNRNPB_avg,18_NONO_avg,20_ALYREF_avg,21_COIL_avg,00_BG488_avg,00_BG568_avg,00_EU_avg,09_SRRM2_ILASTIK_avg,15_SON_ILASTIK_avg
0,373535,plate01,I09,0,0,0,0,6,0,373506,plate01,I09,0,0,0,0,6,0,0.0,NaN,0.0,NaN,0.0,NaN,S,184A1,normal,NaN,43.893030,175.995428,27.577226,54.206169,128.697294,41.469521,60.416069,64.842372,125.169772,65.557846,105.751293,192.479292,78.460671,49.046642,187.326717,99.378451,117.227410,26.282266,60.407161,33.006360,131.946547,28.173045,54.011811,187.774103,131.526724,385.672558,11.862217,81.837107,291.005551,51.128612,368.260323,285.676524,11.584543,8.256946,1.803026,599.449225,8305.377207,10121.675842
1,373536,plate01,I09,0,0,0,0,7,0,373507,plate01,I09,0,0,0,0,7,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,39.837376,273.655677,12.569294,22.001503,81.730022,20.032913,39.631146,44.481310,73.860256,54.789958,67.896671,162.914590,35.864765,37.245010,355.807515,67.943580,76.462250,12.614654,55.775873,14.612852,120.283067,19.574581,41.063924,101.116543,55.055405,297.648758,11.671538,91.374197,329.135675,33.598199,335.386680,410.320858,9.813957,5.888977,1.561608,398.334857,6055.847451,10278.970546
2,373537,plate01,I09,0,0,0,0,8,0,373508,plate01,I09,0,0,0,0,8,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,44.735922,258.942987,23.616477,47.406444,140.684383,44.831516,68.078372,67.289663,85.795525,90.090190,87.339633,249.505730,88.193348,51.833120,332.372822,124.072929,130.945283,38.285344,76.223063,43.566143,151.189717,39.663339,71.273700,205.703836,124.654347,332.255405,17.381051,196.612601,361.317387,55.593623,457.713302,469.516855,14.441616,9.757942,1.859657,464.431683,8280.577392,11757.616541
4,373539,plate01,I09,0,0,0,0,10,0,373510,plate01,I09,0,0,0,0,10,0,0.0,NaN,0.0,NaN,0.0,NaN,S,184A1,normal,NaN,30.892690,231.842236,19.046228,37.117579,100.918987,25.272207,43.387094,43.176593,78.249253,76.013457,112.938592,178.444723,81.792631,41.639270,292.071767,90.931247,99.625565,27.330404,66.549839,20.411372,107.398853,23.828493,55.354041,134.139140,99.027838,301.044591,15.211128,122.575573,334.431894,42.139486,362.906348,438.364101,10.736532,6.967087,1.697280,410.540764,5541.760160,10350.776433
5,373540,plate01,I09,0,0,0,0,11,0,373511,plate01,I09,0,0,0,0,11,0,0.0,NaN,0.0,NaN,0.0,NaN,S,184A1,normal,NaN,37.753066,238.882044,13.789982,27.224439,84.975838,23.843158,40.274607,44.294862,88.214824,56.990949,104.852156,156.820501,104.210400,37.244133,299.586078,84.814062,103.998514,19.334907,53.953275,18.832787,109.794168,22.626681,42.797852,119.819474,68.142357,291.043989,7.073450,92.228603,300.013785,35.563778,306.392659,381.804707,8.969512,6.654304,1.741013,405.846715,6569.049927,8857.680949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,248096,plate01,I09,5,4,0,0,31,0,248051,plate01,I09,5,4,0,0,31,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,34.088505,155.096587,13.024716,28.936674,95.528086,23.275727,43.408660,39.239153,52.576061,40.188708,5.078687,175.720682,51.470518,35.296696,174.323644,56.949624,56.759946,18.220416,41.090229,1

In [10]:
val.metadata

,mapobject_id,plate_name,well_name,well_pos_y,well_pos_x,tpoint,zplane,label,is_border,mapobject_id_cell,plate_name_cell,well_name_cell,well_pos_y_cell,well_pos_x_cell,tpoint_cell,zplane_cell,label_cell,is_border_cell,is_mitotic,is_mitotic_labels,is_polynuclei_HeLa,is_polynuclei_HeLa_labels,is_polynuclei_184A1,is_polynuclei_184A1_labels,cell_cycle,cell_type,perturbation,duration,00_DAPI_avg,07_H2B_avg,01_CDK9_pT186_avg,03_CDK9_avg,05_GTF2B_avg,07_SETD1A_avg,08_H3K4me3_avg,09_SRRM2_avg,10_H3K27ac_avg,11_KPNA2_MAX_avg,12_RB1_pS807_S811_avg,13_PABPN1_avg,14_PCNA_avg,15_SON_avg,16_H3_avg,17_HDAC3_avg,19_KPNA1_MAX_avg,20_SP100_avg,21_NCL_avg,01_PABPC1_avg,02_CDK7_avg,03_RPS6_avg,05_Sm_avg,07_POLR2A_avg,09_CCNT1_avg,10_POL2RA_pS2_avg,11_PML_avg,12_YAP1_avg,13_POL2RA_pS5_avg,15_U2SNRNPB_avg,18_NONO_avg,20_ALYREF_avg,21_COIL_avg,00_BG488_avg,00_BG568_avg,00_EU_avg,09_SRRM2_ILASTIK_avg,15_SON_ILASTIK_avg
15,373552,plate01,I09,0,0,0,0,23,0,373523,plate01,I09,0,0,0,0,23,0,0.0,NaN,0.0,NaN,0.0,NaN,S,184A1,normal,NaN,30.574293,249.569570,7.583841,12.275772,68.128334,10.414950,17.431491,26.870564,56.546820,49.877291,48.190328,131.746804,65.050555,31.259364,313.478391,64.566879,95.895774,13.130200,44.428449,13.922616,57.981341,17.937401,40.632685,47.944492,34.554776,235.647705,7.176806,84.357927,310.685763,29.629622,240.284502,347.773641,7.038823,4.764324,1.504008,348.361510,3933.958062,9260.937295
17,228069,plate01,I09,0,1,0,0,2,0,228048,plate01,I09,0,1,0,0,2,0,0.0,NaN,0.0,NaN,0.0,NaN,S,184A1,normal,NaN,37.439699,245.274340,17.889401,28.128698,77.139874,18.420196,29.043601,50.530310,66.000257,91.896682,79.542296,141.386941,99.579891,37.320648,280.587424,92.903227,111.034924,14.532252,53.764492,22.844649,112.060962,27.840653,55.110393,108.570324,85.880647,311.581509,13.488006,88.534204,349.458715,38.682724,302.884503,379.881793,10.144150,7.706533,1.765122,435.851395,6589.405269,9630.423705
41,291043,plate01,I09,0,3,0,0,14,0,290977,plate01,I09,0,3,0,0,14,0,0.0,NaN,0.0,NaN,0.0,NaN,S,184A1,normal,NaN,40.567000,163.835686,19.704372,35.336253,107.740268,36.216175,45.366358,57.972970,73.875981,61.487103,145.352718,155.916906,69.674045,42.145291,177.018449,65.471572,54.560807,18.421989,46.049133,16.959562,147.944396,19.154870,40.642219,144.793910,90.021095,293.602179,12.477208,94.965482,213.413462,38.985342,313.432754,273.643137,9.176985,6.443995,1.671419,412.074239,8498.392042,11110.759694
50,291056,plate01,I09,0,3,0,0,27,0,290990,plate01,I09,0,3,0,0,27,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,29.099963,162.728813,11.833903,21.418495,101.488400,26.886086,30.653287,47.203884,37.454291,33.303338,3.974910,176.756954,50.318569,38.093269,188.648134,54.823196,42.768541,10.717219,28.983672,13.584760,103.585063,13.901486,35.749202,121.093297,57.866152,270.137450,10.363075,96.477406,239.218251,29.967979,331.050846,305.319775,9.088626,5.134492,1.673023,367.283281,7519.626184,10119.844923
57,291065,plate01,I09,0,3,0,0,36,0,290999,plate01,I09,0,3,0,0,36,0,0.0,NaN,0.0,NaN,0.0,NaN,G2,184A1,normal,NaN,41.026765,148.126392,18.295315,35.514461,100.458513,33.749825,53.693365,57.645552,86.948187,65.771916,99.316916,176.841702,60.806458,36.446176,163.000022,64.563060,55.952194,24.514409,51.765313,19.439434,134.295217,19.637452,43.949357,162.218134,89.970926,275.154056,12.679415,110.516304,220.748026,42.265815,313.468876,274.557672,8.199510,6.213666,1.587850,437.427554,6760.496931,9142.113985
62,291072,plate01,I09,0,3,0,0,43,0,291006,plate01,I09,0,3,0,0,43,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,30.452216,166.752829,12.606276,21.299407,98.027614,30.724684,35.064603,46.082949,49.667228,36.128957,2.462416,184.807542,45.589469,40.229341,187.956096,60.462727,48.111862,11.873443,30.825990,14.291511,109.049414,16.139468,38.152107,115.967548,60.687516,258.036370,8.661483,86.111249,240.142465,33.724408,332.029395,287.157795,9.585442,5.725089,1.671830,390.576079,6154.798476,9207.179740
65,291075,plate01,I09,0,3,0,0,46,0,291009,plate01,I09,0,3,0,0,46,0,0.0,NaN,0.0,NaN

In [11]:
test.metadata

,mapobject_id,plate_name,well_name,well_pos_y,well_pos_x,tpoint,zplane,label,is_border,mapobject_id_cell,plate_name_cell,well_name_cell,well_pos_y_cell,well_pos_x_cell,tpoint_cell,zplane_cell,label_cell,is_border_cell,is_mitotic,is_mitotic_labels,is_polynuclei_HeLa,is_polynuclei_HeLa_labels,is_polynuclei_184A1,is_polynuclei_184A1_labels,cell_cycle,cell_type,perturbation,duration,00_DAPI_avg,07_H2B_avg,01_CDK9_pT186_avg,03_CDK9_avg,05_GTF2B_avg,07_SETD1A_avg,08_H3K4me3_avg,09_SRRM2_avg,10_H3K27ac_avg,11_KPNA2_MAX_avg,12_RB1_pS807_S811_avg,13_PABPN1_avg,14_PCNA_avg,15_SON_avg,16_H3_avg,17_HDAC3_avg,19_KPNA1_MAX_avg,20_SP100_avg,21_NCL_avg,01_PABPC1_avg,02_CDK7_avg,03_RPS6_avg,05_Sm_avg,07_POLR2A_avg,09_CCNT1_avg,10_POL2RA_pS2_avg,11_PML_avg,12_YAP1_avg,13_POL2RA_pS5_avg,15_U2SNRNPB_avg,18_NONO_avg,20_ALYREF_avg,21_COIL_avg,00_BG488_avg,00_BG568_avg,00_EU_avg,09_SRRM2_ILASTIK_avg,15_SON_ILASTIK_avg
3,373538,plate01,I09,0,0,0,0,9,0,373509,plate01,I09,0,0,0,0,9,0,0.0,NaN,0.0,NaN,0.0,NaN,S,184A1,normal,NaN,43.931378,194.444293,18.353704,32.399117,104.564078,29.361877,46.617407,51.506499,127.183208,55.535536,94.269486,171.038290,77.086018,32.756867,212.862917,74.396402,81.701189,15.256486,49.931726,19.315464,136.797350,21.712636,45.183902,148.422635,82.323469,284.648928,8.077076,50.190543,271.552491,37.614959,285.612350,263.922204,7.988940,6.477020,1.717521,425.533338,7441.930589,8530.446240
7,373543,plate01,I09,0,0,0,0,14,0,373514,plate01,I09,0,0,0,0,14,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,31.796757,167.421060,8.088238,16.159192,54.151197,14.970476,34.413437,34.801339,45.406018,42.710192,2.001603,133.262292,15.262286,28.887519,165.452414,56.010599,75.013360,15.859371,36.431576,17.900580,65.072087,19.961104,36.627070,48.509038,41.970375,245.626208,13.282367,65.685411,237.447556,31.389509,291.177621,247.545313,7.859336,6.561661,1.630393,372.463443,5621.383479,8108.554629
9,373546,plate01,I09,0,0,0,0,17,0,373517,plate01,I09,0,0,0,0,17,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,23.068771,167.207771,14.759187,19.036149,126.775173,31.242133,33.646919,49.393888,35.833057,77.340375,2.787389,167.636919,56.760069,34.700280,179.047911,92.821322,97.650617,24.061671,55.531707,23.159593,142.841748,24.482600,53.103061,127.423264,69.501830,246.930370,13.879289,144.387664,232.434658,44.611408,351.037382,327.513410,8.695239,7.581144,1.645540,372.811467,6717.641716,8216.274439
24,228080,plate01,I09,0,1,0,0,13,0,228059,plate01,I09,0,1,0,0,13,0,0.0,NaN,0.0,NaN,0.0,NaN,G2,184A1,normal,NaN,32.078867,178.269895,18.796561,33.388943,107.821185,41.319326,48.081123,55.600895,72.199884,74.079468,129.651423,203.698225,63.982787,45.901579,233.402337,93.077872,106.523702,21.240588,64.664777,34.699882,157.854332,30.853312,53.062213,160.139889,104.704105,288.536858,11.130135,130.600651,280.562619,50.338534,357.159856,377.947482,10.856259,7.269352,1.851786,479.620915,7868.494900,10684.152555
28,324142,plate01,I09,0,2,0,0,7,0,324083,plate01,I09,0,2,0,0,7,0,0.0,NaN,0.0,NaN,0.0,NaN,G2,184A1,normal,NaN,44.098693,169.027253,13.698369,25.617715,84.534710,25.511925,57.734848,33.060869,124.012409,54.870709,99.773236,142.218122,50.510787,29.267737,219.390641,60.383234,53.727743,16.020160,43.992535,18.459753,113.135804,18.367440,36.470982,116.697646,49.731282,253.758183,7.708204,66.395213,228.104237,34.602493,236.486539,291.986443,7.262463,5.789343,1.550700,361.682005,5776.833068,8166.957938
38,291040,plate01,I09,0,3,0,0,11,0,290974,plate01,I09,0,3,0,0,11,0,0.0,NaN,0.0,NaN,0.0,NaN,G1,184A1,normal,NaN,26.740385,128.608741,15.477700,35.398259,93.286975,30.849339,40.518885,47.552590,38.435648,39.691790,2.440736,172.727960,43.155923,30.697981,129.359626,60.788100,50.260814,31.341237,50.148643,18.475380,79.774650,20.211609,48.552420,129.459433,95.528248,221.968939,12.843103,114.028953,179.354214,43.021642,276.596443,236.118496,7.764290,6.579578,1.685376,375.378452,6069.066786,8316.465715
69,383787,plate01,I09,0,4,0,0,13,0,383751,plate01,I09,0,4,0,0,13,0,0.0,NaN,0.0,NaN,

## Save data

Prepare to save data:

In [12]:
p.keys()

dict_keys(['_comment', 'raw_data_dir', 'dir_type', 'perturbations_and_wells', 'output_data_dir', 'seed', 'input_channels', 'output_channels', 'aggregate_output', 'train_frac', 'val_frac', 'img_size', 'subtract_background', 'background_value', 'normalise', 'percentile', 'add_cell_cycle_to_metadata', 'cell_cycle_file', 'add_well_info_to_metadata', 'well_info_file', 'filter_criteria', 'filter_values', 'convert_into_image', 'remove_original_data', 'project_into_scalar', 'method', 'parameter_file_name', 'data_dirs', 'normalise_rescale_values'])

In [13]:
import shutil

# create dir
outdir = p['output_data_dir']
if os.path.exists(outdir):
    print('Warning! Directory {} already exist! Deleting...\n'.format(outdir))
    try:
        shutil.rmtree(outdir)
    except OSError as e:
        print('Dir {} could not be deleted!\n\nOSError: {}'.format(outdir, e))

print('Creating dir: {}'.format(outdir))
os.makedirs(outdir, exist_ok=False)
    

Warning! Directory /home/hhughes/Documents/Master_Thesis/Project/datasets/184A1_hannah_EU_regression_TEST_2 already exist! Deleting...

Creating dir: /home/hhughes/Documents/Master_Thesis/Project/datasets/184A1_hannah_EU_regression_TEST_2


In [14]:
# Get channels ids (proteins) which will be used to predict transcripcion rate
input_ids = list(train.channels.set_index('name').loc[p['input_channels']]['channel_id'])
# Get id of the channel that measure trancripcion rate
output_ids = list(train.channels.set_index('name').loc[p['output_channels']]['channel_id'])
# add output channel id after the input channels ids
channels_ids = input_ids + output_ids

Save metadata and used parameters

In [15]:
# save params
json.dump(p, open(os.path.join(outdir, 'params.json'), 'w'), indent=4)

# save metadata
train.metadata.to_csv(os.path.join(outdir, 'train_metadata.csv'))
val.metadata.to_csv(os.path.join(outdir, 'val_metadata.csv'))
test.metadata.to_csv(os.path.join(outdir, 'test_metadata.csv'))
pd.concat([train.metadata, val.metadata, test.metadata]).to_csv(os.path.join(outdir, 'metadata.csv'))

# Save used channels
#train.channels.to_csv(os.path.join(outdir, 'channels.csv'))
train.channels.set_index('channel_id').loc[channels_ids].to_csv(os.path.join(outdir, 'channels.csv'))

Save Images

In [16]:
# Note! instead of calculating the response value (y) here and save
# it separatelly, instead we will do it on the modeling part

"""
# get images
train_dataset = np.array(train.get_object_imgs(data='MPP', img_size=p['img_size']))
del(train)
val_dataset = np.array(val.get_object_imgs(data='MPP', img_size=p['img_size']))
del(val)
test_dataset = np.array(test.get_object_imgs(data='MPP', img_size=p['img_size']))
del(test)

# Create responce variable (y)
if p['aggregate_output'] == 'avg':
    train_dataset_y = np.array([img[img!=0].mean() for img in train_dataset[:,:,:,output_ids]])
    val_dataset_y = np.array([img[img!=0].mean() for img in val_dataset[:,:,:,output_ids]])
    test_dataset_y = np.array([img[img!=0].mean() for img in test_dataset[:,:,:,output_ids]])

# Save datasets
np.savez(os.path.join(outdir, 'train_dataset.npz'), x=train_dataset[:,:,:,input_ids], y=train_dataset_y)
del(train_dataset)
np.savez(os.path.join(outdir, 'val_dataset.npz'), x=val_dataset[:,:,:,input_ids], y=val_dataset_y)
del(val_dataset)
np.savez(os.path.join(outdir, 'test_dataset.npz'), x=test_dataset[:,:,:,input_ids], y=test_dataset_y)
del(test_dataset)
"""

if p['convert_into_image']:
    # get images and mask, save them and delete vars
    print('Saving train images and masks...')
    np.save(os.path.join(outdir, 'train_images.npy'), train.images[:,:,:,channels_ids])
    del(train.images)
    np.save(os.path.join(outdir, 'train_mask.npy'), train.masks)
    del(train.masks)
    del(train)

    print('Saving validation images and masks...')
    np.save(os.path.join(outdir, 'val_images.npy'), val.images[:,:,:,channels_ids])
    del(val.images)
    np.save(os.path.join(outdir, 'val_mask.npy'), val.masks)
    del(val.masks)
    del(val)

    print('Saving test images and masks...')
    np.save(os.path.join(outdir, 'test_images.npy'), test.images[:,:,:,channels_ids])
    del(test.images)
    np.save(os.path.join(outdir, 'test_mask.npy'), test.masks)
    del(test.masks)
    del(test)